<h1>STORED PROCEDURES</h1>

<h3>Create sp_validate_incoming_msg</h3>
<p><span style="font-size: 12pt;">This stored procedure will check if the message is valid. All common checking routines are coded here.</span></p>

In [ ]:
create PROCEDURE sp_validate_incoming_msg(
	IN 	cellno varchar(11), 
	IN 	msg varchar(65000),
	OUT is_valid tinyint)
BEGIN
	
declare cmdname varchar(20) default '';
set cmdname = (select split(msg, ' ',1));
	
if not exists(select * from e_subscribers es where es.cellno = cellno) then
	-- do not send a sms response.
	set is_valid=0;
elseif not exists(select * from e_commands ec where ec.name = cmdname) then
 	insert into e_outbox (id,cellno, message) select uuid(), cellno, concat('Invalid command name ',cmdname,'.'); 
 	set is_valid=0;
else
	set is_valid=1;
end if;

END;

<h3 id="Create-sp_cmd_reg">Create sp_cmd_reg</h3>
<p><span style="font-size: 12pt;">This stored procedure will process the command registration. The SMS format for this command is <span style="color: #236fa1;"><strong>REG</strong></span><span style="color: #b96ad9;">&lt;space&gt;</span><span style="color: #236fa1;"><strong>LName/FName/M</strong>I</span><span style="color: #b96ad9;">&lt;space&gt;</span><span style="color: #236fa1;"><strong>CellNoToRegister</strong></span></span></p>

In [ ]:
create PROCEDURE sp_cmd_reg(
	IN 	e_inbox_id char(36),
	IN 	cellno varchar(11), 
	IN 	msg varchar(65000),
	OUT is_valid tinyint)
BEGIN

declare cmdname varchar(20) default '';
declare fullname varchar(100) default '';
declare cellno_reg varchar(11) default '';
declare lastname varchar(100) default '';
declare firstname varchar(100) default '';
declare mi varchar(100) default '';
declare upline_id char(36) default '';

START TRANSACTION;

set cmdname = (select split(msg, ' ',1));
set fullname = (select split(msg, ' ',2));
set cellno_reg = (select split(msg, ' ',3));
set lastname = (select split(fullname, '/',1));
set firstname = (select split(fullname, '/',2));
set mi = (select split(fullname, '/',3));
set upline_id = (select id from e_subscribers es where es.cellno = cellno );

if exists(select es.* from e_subscribers es where es.cellno = cellno_reg) then
	-- send a response that the number is already registered.
	insert into e_outbox (id,cellno, message) select uuid(), cellno, concat('The cellno ',cellno_reg, ' is already registed.'); 
else
	insert into e_subscribers (id, upline_id, cellno, last_name, first_name, mi) select uuid(), upline_id, cellno_reg, lastname, firstname, mi;
	insert into e_outbox (id, e_inbox_id, cellno , message) select uuid(), e_inbox_id, cellno, concat('You have succesfully registered ',lastname, ',',firstname,' ',mi, ' with cellno ',cellno_reg) ; 
end if;

COMMIT;
END;

<p><span style="font-size: 12pt; color: #e03e2d;">NOTE: Make sure that the trigger<strong><em> <span style="text-decoration: underline;">tr_after_insert_inbox_process_cmd</span></em></strong> has been created.</span></p>

<p><span style="font-size: 12pt;">Let's put it in action... Execute an insert statement in table e_inbox so that the trigger tr_after_insert_inbox_process_cmd will execute.</span></p>

In [ ]:
insert into e_inbox (id, cellno, message) select uuid(), '09222853161', 'REG SAAVEDRA/JOHN/D 09178553218'

<p><span style="font-size: 12pt;">Validate the tables affected if the process is successful.</span></p>

In [ ]:
select * from e_subscribers 
select * from e_inbox

<h3>Create sp_cmd_transfer</h3>
<p><span style="font-size: 12pt;">This stored procedure will transfer funds to the registered subscribers.</span></p>

In [ ]:
create PROCEDURE sp_cmd_transfer(
	IN 	e_inbox_id char(36),
	IN 	cellno varchar(11), 
	IN 	msg varchar(65000),
	OUT is_valid tinyint)
BEGIN

declare cmdname varchar(20) default '';
declare recipient varchar(11) default '';
declare recipient_id char(36) default '';
declare recipient_name varchar(100) default '';
declare amount decimal(15,2) default 0.0;
declare subscribers_id char(36) default '';
declare subscribers_name varchar(100) default '';
declare e_command_id char(36) default '';


-- START TRANSACTION;

set cmdname = (select split(msg, ' ',1));
set recipient = (select split(msg, ' ',2));
set recipient_id = (select id from e_subscribers es where es.cellno = recipient);
set recipient_name = (select concat(es.last_name,',',es.first_name,' ', es.mi,'.') from e_subscribers es where es.cellno = recipient);
set amount = (select split(msg, ' ',3));
set subscribers_id = (select id from e_subscribers es where es.cellno = cellno);
set subscribers_name = (select concat(es.last_name,',',es.first_name,' ', es.mi,'.') from e_subscribers es where es.cellno = cellno);
set e_command_id = (select id from e_commands ec where ec.name = cmdname);

if not exists(select es.* from e_subscribers es where es.cellno = recipient) then
	-- send a response that the number is not a registered subscriber.
	insert into e_outbox (id,cellno, message) select uuid(), cellno, concat('Unable to transfer funds to ',recipient, ', because it is not a registered subscriber.'); 
else
	-- process your account.
	insert into e_ledger (id,e_subscribers_id,e_inbox_id,e_command_id,amount_beginning,amount_actual,document_date) 
	select uuid(),subscribers_id, e_inbox_id, e_command_id, sf_get_current_balance(cellno), amount*-1, now();
	insert into e_outbox (id, e_inbox_id, cellno , message) select uuid(), e_inbox_id, cellno, concat('You have succesfully transfered an amount of ',amount, ' to ',recipient,' - ',recipient_name);
	-- process recipient's account
	insert into e_ledger (id,e_subscribers_id,e_inbox_id,e_command_id,amount_beginning,amount_actual,document_date) 
	select uuid(),recipient_id, e_inbox_id, e_command_id, sf_get_current_balance(recipient), amount, now();
	insert into e_outbox (id, e_inbox_id, cellno , message) select uuid(), subscribers_id, cellno, concat('You have received a fund transfer from ',subscribers_name, ' (',cellno,') with an amount of ',amount,'.');
end if;

-- COMMIT;
END;

<p><span style="font-size: 16px;">Let's put it in action... Execute an insert statement in table e_inbox so that the trigger tr_after_insert_inbox_process_cmd will execute.</span></p>

In [ ]:
insert into e_inbox (id, cellno, message) select uuid(), '09222853161', 'TRANSFER 09201632077 5000'

<p><span style="font-size: 16px;">Validate the tables affected if the process is successful.</span></p>

In [ ]:
select * from e_inbox
select * from e_outbox eo 
select * from e_ledger el 